<a href="https://colab.research.google.com/github/SaiDunoyer/Apriori/blob/master/Doc_classification_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
import os

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# avg w = 2481 average h = 3509 # 1200, 1750
img_width, img_height = 372, 532

base_path = '/content/gdrive/My Drive/Colab Notebooks/classifier_dataset_v4'
train_data_dir = os.path.join(base_path, 'train')
labels = [label for label in os.listdir(train_data_dir)]
validation_data_dir = os.path.join(base_path, 'test')
nb_train_samples = sum([len(os.listdir(os.path.join(train_data_dir, label))) for label in labels])
nb_validation_samples = sum([len(os.listdir(os.path.join(validation_data_dir, label))) for label in labels])

for label in labels:
    for png_file in os.listdir(os.path.join(train_data_dir, label)):
        X = image.load_img(os.path.join(train_data_dir, label, png_file), target_size=(img_width, img_height))
        X = image.img_to_array(X)
        X = preprocess_input(X)

epochs = 10
batch_size = 10

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(12))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size, class_mode='binary')

print('train', train_generator.class_indices)
weights = {v: 1 - len(os.listdir(os.path.join(train_data_dir, k))) / nb_train_samples for k,v in train_generator.class_indices.items()}
print('weights', weights)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size, class_mode='binary')

print('test', validation_generator.class_indices)



checkpoint = ModelCheckpoint(os.path.join(base_path, 'best_model_xs_v4.h5'), monitor='loss', verbose=1,
                             save_best_only=True, mode='auto', save_freq=1)

model.fit(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs, validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,
                    class_weight=weights,
                    callbacks=[checkpoint])

model.save_weights(os.path.join(base_path, 'model_saved_xs_v4.h5'))

with open(os.path.join(base_path, 'model_saved_xs_v4.json'), 'w') as f:
    f.write(model.to_json())


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Found 1362 images belonging to 9 classes.
train {'bil': 0, 'ce_sp': 1, 'doc_id': 2, 'irap': 3, 'other': 4, 'pf': 5, 'ric_cc': 6, 'ric_unico': 7, 'sp': 8}
weights {0: 0.9904552129221733, 1: 0.9588839941262849, 2: 0.711453744493392, 3: 0.9654919236417033, 4: 0.5778267254038179, 5: 0.9405286343612335, 6: 0.9243759177679882, 7: 0.9691629955947136, 8: 0.9618208516886931}
Found 243 images belonging to 9 classes.
test {'bil': 0, 'ce_sp': 1, 'doc_id': 2, 'irap': 3, 'other': 4, 'pf': 5, 'ric_cc': 6, 'ric_unico': 7, 'sp': 8}
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/10
  1/136 [..............................] - ETA: 48:28 - loss: 44.8923 - accuracy: 0.0000e+00
Epoch 00001: loss improved from inf to 44.89225, saving model to /content/gdrive/My Drive/Colab Notebooks/classifier_dataset_v4/best_model_xs_v4.h5
  2/136 [..............................] - ETA: 10:13 - loss: 415.8456 - accuracy: 0.0750